In [28]:
import pandas as pd
import os

In [30]:
DATA_FOLDER="data_final/"

In [17]:
 class LemmatizeTextCorp(object):
    def __init__(self):
        from pymystem3 import Mystem
        import re
        self.lemmatizer = Mystem()
        self.find_label = re.compile(r'([A-Z]+)(,|=)')
        self.vocab={'A': 'ADJ','ADV': 'ADV','ADVPRO': 'ADV',\
                    'ANUM': 'ADJ','APRO': 'DET','COM': 'ADJ',\
                    'CONJ': 'SCONJ','INTJ': 'INTJ',\
                    'NONLEX': 'X','NUM': 'NUM','PART': 'PART',\
                    'PR': 'ADP','S': 'NOUN','SPRO': 'PRON',\
                    'UNKN': 'X','V': 'VERB'}
        self.get_lex = lambda x: x['analysis'][0]['lex']
        self.get_label = lambda x: self.vocab[self.find_label.search(x['analysis'][0]['gr']).group(1)]
    def fit(self,X,y=None):
        return self
    def transform(self,X,y=None):
        result=[]
#         from IPython.core.debugger import Pdb; Pdb().set_trace()
        for line in X:
            line = self.lemmatizer.analyze(line)
            result.append(["%s_%s"%(self.get_lex(el),self.get_label(el))\
                           for el in line if 'analysis' in el and el['analysis']])
        return result

In [18]:
lm = LemmatizeTextCorp()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [6]:
import gensim.models

In [9]:
model = gensim.models.KeyedVectors.load_word2vec_format("model/model.bin",binary=True)

In [94]:
model1 = gensim.models.FastText.load_fasttext_format("model1.bin")

In [100]:
model1.wv.most_similar("PSP")

[('PS3', 0.9861921668052673),
 ('PS4', 0.9858267307281494),
 ('PSN', 0.976943850517273),
 ('Move"', 0.9664517045021057),
 ('SCEE)', 0.963442325592041),
 ('PS', 0.9515871405601501),
 ('+', 0.9513316750526428),
 ('Move', 0.9486615657806396),
 ('«Sony', 0.9396472573280334),
 ('(Dualshock', 0.9381834268569946)]

In [26]:
import fasttext

In [91]:
model = fasttext.skipgram('data.txt', 'model1',dim=100,minn=2,maxn=5)

In [92]:
model.words

{'Beatles',
 'Evil',
 '(код',
 'Warrior',
 'Lord',
 'учит',
 'подарочная',
 'Mortal',
 'музыкальный',
 'Стэн',
 '[РС,',
 'УЛИЦЕ',
 'Pin',
 'Снайпер',
 'Ice',
 'Hit',
 '-',
 'It',
 'Сергей,',
 'Xenomorph',
 '(сериал)',
 'Декабрь',
 'Сергей',
 'МОНСТРОВ.',
 'ВЫСОКОЕ',
 'Raider.',
 '(BD+DVD)',
 'Strike',
 'Caf?',
 'том',
 'мелодий',
 'Палка',
 'Tails',
 'Run',
 '(rem.)',
 'Raynor',
 'BUUREN',
 'песни',
 'геймпада',
 '(Строения',
 '(CD-Digipack)',
 'СНОВ',
 'СИСТЕМЫ',
 'СТУЛЬЕВ',
 'ЧУВСТВА',
 '6',
 'Петров',
 'case',
 ')',
 'Август',
 'ЗЛА',
 '6.X',
 'LARA',
 '1С:Специалист',
 'Vita,',
 'набор,',
 'Square',
 'EMTEC',
 'ДО',
 'игрушка',
 'Война',
 'Тени',
 'двух',
 'ДЖЕКСОН',
 'Heavy',
 'Radio',
 'А.С.',
 'New',
 'Tour',
 'Regular',
 'KITTY',
 'Jones',
 'версия)',
 'Civilization:',
 'ПАРНИ',
 'Dogs.',
 'контроллер',
 '(фирм)',
 'КВ-1С',
 'Box',
 'ЖЕНЩИНЫ',
 'пазлы',
 'МЕНЯ',
 '[PC,',
 '(a',
 'Земля',
 'From',
 'времена',
 '23',
 'СПУСТЯ',
 'ДАЛМАТИНЕЦ',
 'РЕЙС',
 'Марк',
 'ЛЕОПОЛЬДА',
 'Und

# Reading Data 

### Прочитаем данные

In [31]:
transactions    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv.gz'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))
smpsb_df = pd.read_csv(os.path.join(
    DATA_FOLDER,
    "sample_submission.csv.gz"),
    header=0, sep=',', quotechar='"')

In [32]:
test_df = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv.gz'))

In [21]:
transactions.date=pd.to_datetime(transactions.date,format='%d.%m.%Y')
transactions['day']= transactions.date.dt.day
transactions['month'] = transactions.date.dt.month
transactions['year'] = transactions.date.dt.year

In [22]:
transactions.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year
0,2013-01-02,0,59,22154,999.00,1.0,2,1,2013
1,2013-01-03,0,25,2552,899.00,1.0,3,1,2013
2,2013-01-05,0,25,2552,899.00,-1.0,5,1,2013
3,2013-01-06,0,25,2554,1709.05,1.0,6,1,2013
4,2013-01-15,0,25,2555,1099.00,1.0,15,1,2013


In [28]:
transactions[['shop_id','item_id','item_price','item_cnt_day']].describe()

,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [90]:
items.item_name.to_csv("data.txt",encoding="utf-8")

In [83]:
items.item_name

0                ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D
1        !ABBYY FineReader 12 Professional Edition Full...
2            ***В ЛУЧАХ СЛАВЫ   (UNV)                    D
3          ***ГОЛУБАЯ ВОЛНА  (Univ)                      D
4              ***КОРОБКА (СТЕКЛО)                       D
5        ***НОВЫЕ АМЕРИКАНСКИЕ ГРАФФИТИ  (UNI)         ...
6                 ***УДАР ПО ВОРОТАМ (UNI)               D
7               ***УДАР ПО ВОРОТАМ-2 (UNI)               D
8                 ***ЧАЙ С МУССОЛИНИ                     D
9            ***ШУГАРЛЭНДСКИЙ ЭКСПРЕСС (UNI)             D
10               *ЗА ГРАНЬЮ СМЕРТИ                       D
11               *ЛИНИЯ СМЕРТИ                           D
12                         *МИХЕЙ И ДЖУМАНДЖИ  Сука любовь
13               *СПАСАЯ ЭМИЛИ                           D
14               *ЧОКНУТЫЙ ПРОФЕССОР /МАГИЯ/             D
15                  //АДРЕНАЛИН: ОДИН ПРОТИВ ВСЕХ (Регион)
16                              //МОНГОЛ С.Бодров (Регио

In [82]:
item_categories

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4
5,Аксессуары - PSVita,5
6,Аксессуары - XBOX 360,6
7,Аксессуары - XBOX ONE,7
8,Билеты (Цифра),8
9,Доставка товара,9


In [25]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4
